<a href="https://colab.research.google.com/github/ricardoruedas/ML/blob/main/%5B05%5D%20-%20Arboles%20de%20decision/Gradient_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arboles de decisión: Gradient Boosting - Ejercicio 2: Gradient_Boosting.ipynb

Este notebook es un **I do**: todo resuelto y explicado paso a paso.

## Objetivos

- Creo un dataset alearotio con 1000 ejemplos de emails para detector de SPAM
- Entrena un Gradient Boosting.
- Evalúa el modelo con métricas de clasificación (accuracy, matriz de confusión y reporte).
- Muestra la importancia de cada característica (qué variables usa más el modelo para decidir).

## 1) Cargamos librerias clasificacion y regresion

In [2]:
# cargo librerias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import accuracy_score, mean_squared_error
import matplotlib.pyplot as plt


# La filosofia del Gradient Boosting

print("LA FILOSOFÍA DE GRADIENT BOOSTING:")
print("-" * 40)
print("📚 Random Forest: '¿Qué opinan 100 expertos?'")
print("🎯 Gradient Boosting: '¿Cómo mejoro mi predicción paso a paso?'")
print("\n🔄 Proceso:")
print("   Modelo 1 → Errores → Modelo 2 → Errores → Modelo 3 → ...")

LA FILOSOFÍA DE GRADIENT BOOSTING:
----------------------------------------
📚 Random Forest: '¿Qué opinan 100 expertos?'
🎯 Gradient Boosting: '¿Cómo mejoro mi predicción paso a paso?'

🔄 Proceso:
   Modelo 1 → Errores → Modelo 2 → Errores → Modelo 3 → ...


## 2) Descripción del dataset (Ejemplo de clasificacion)

El dataset de emails contiene 1000 muestras de emails potencialmente sospechosos, creados de manera random, dividido en 5 categorias diferentes.
    'palabras_sospechosas',
    'signos_exclamacion',
    'enlaces_externos',
    'mayusculas_porcentaje',
    'longitud_email'

In [3]:
# Crear datos simulados de emails
np.random.seed(42)
X_clf, y_clf = make_classification(
    n_samples=1000,
    n_features=5,
    n_informative=3,
    n_redundant=1,
    random_state=42
)

# Nombres más intuitivos para las características
feature_names = [
    'palabras_sospechosas',
    'signos_exclamacion',
    'enlaces_externos',
    'mayusculas_porcentaje',
    'longitud_email'
]

df_emails = pd.DataFrame(X_clf, columns=feature_names)
df_emails['es_spam'] = y_clf

print("📊 Primeros 5 emails:")
print(df_emails.head())

📊 Primeros 5 emails:
   palabras_sospechosas  signos_exclamacion  enlaces_externos  \
0             -0.038769           -0.649239         -0.224746   
1              1.005284           -1.373239          1.157346   
2             -0.742455           -0.573257          1.688442   
3             -1.587158            1.758582         -0.930664   
4              0.195806           -0.058897         -0.549360   

   mayusculas_porcentaje  longitud_email  es_spam  
0              -1.346275        0.126879        0  
1               0.126493        1.422799        0  
2              -2.588237        0.762562        0  
3               0.764614        2.415399        1  
4               0.777375        1.147261        1  


## 3) Dividimos train y test

In [4]:
# Dividir datos al 30%
X_train, X_test, y_train, y_test = train_test_split(
    X_clf, y_clf, test_size=0.3, random_state=42
)

## 4) Cargamos el Gradient Boosting Classifier

In [5]:
print("\n ENTRENANDO GRADIENT BOOSTING:")
print("-" * 40)

# Crear el modelo
gbm_clf = GradientBoostingClassifier(
    n_estimators=50,        # 50 árboles secuenciales
    learning_rate=0.1,      # Qué tanto aprende en cada paso
    max_depth=3,           # Árboles pequeños (weak learners)
    random_state=42
)

print("⚙️  Configuración:")
print(f"   🌳 Número de árboles: {gbm_clf.n_estimators}")
print(f"   📚 Tasa de aprendizaje: {gbm_clf.learning_rate}")
print(f"   📏 Profundidad máxima: {gbm_clf.max_depth}")

# Entrenar
print("\n🔄 Entrenando modelo...")
gbm_clf.fit(X_train, y_train)

# Predicciones
y_pred = gbm_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"✅ Precisión obtenida: {accuracy:.3f} ({accuracy*100:.1f}%)")



 ENTRENANDO GRADIENT BOOSTING:
----------------------------------------
⚙️  Configuración:
   🌳 Número de árboles: 50
   📚 Tasa de aprendizaje: 0.1
   📏 Profundidad máxima: 3

🔄 Entrenando modelo...
✅ Precisión obtenida: 0.940 (94.0%)


## 5) Como aprende el modelo, resultados

In [9]:
print("\n VIENDO CÓMO APRENDE PASO A PASO:")
print("-" * 50)

# Calcular error por cada árbol añadido
train_scores = gbm_clf.train_score_
test_scores = []

for i in range(1, gbm_clf.n_estimators + 1):
    # Crear modelo con solo i árboles
    temp_model = GradientBoostingClassifier(
        n_estimators=i,
        learning_rate=gbm_clf.learning_rate,
        max_depth=gbm_clf.max_depth,
        random_state=42
    )
    temp_model.fit(X_train, y_train)
    pred = temp_model.predict(X_test)
    test_scores.append(accuracy_score(y_test, pred))

# Mostrar evolución
print("📈 Evolución del modelo:")
for i in [1, 10, 20, 30, 40, 50]:
    if i <= len(test_scores):
        print(f"   Después de {i:2d} árboles: {test_scores[i-1]:.3f}")



 VIENDO CÓMO APRENDE PASO A PASO:
--------------------------------------------------
📈 Evolución del modelo:
   Después de  1 árboles: 0.913
   Después de 10 árboles: 0.950
   Después de 20 árboles: 0.940
   Después de 30 árboles: 0.940
   Después de 40 árboles: 0.940
   Después de 50 árboles: 0.940


## 6) Descripción del dataset (Ejemplo de regresión)

El dataset de emails contiene 500 muestras de precios de casas, dividido en 4 categorias diferentes.
    'metros_cuadrados',
    'num_habitaciones',
    'antiguedad',
    'distancia_centro'

In [10]:
print("\n EJEMPLO DE REGRESIÓN: PRECIOS DE CASAS")
print("-" * 50)

# Datos simulados de casas
X_reg, y_reg = make_regression(
    n_samples=500,
    n_features=4,
    noise=0.1,
    random_state=42
)

house_features = [
    'metros_cuadrados',
    'num_habitaciones',
    'antiguedad',
    'distancia_centro'
]

df_casas = pd.DataFrame(X_reg, columns=house_features)
df_casas['precio'] = y_reg

print("🏠 Primeras 5 casas:")
print(df_casas.head())


 EJEMPLO DE REGRESIÓN: PRECIOS DE CASAS
--------------------------------------------------
🏠 Primeras 5 casas:
   metros_cuadrados  num_habitaciones  antiguedad  distancia_centro     precio
0         -0.923233         -1.406661   -0.611518         -1.351685 -81.342131
1         -1.344451         -0.281785   -0.420187         -0.918652 -75.999739
2         -2.067442         -0.032695    0.384065         -0.089120 -57.612023
3          0.307407         -0.276813   -0.221254          0.815737   8.807619
4          0.916328         -1.998201    0.078635          0.346488  25.887176


## 7) Dividimos train y test

In [12]:
# Dividir datos
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)

## 8) Cargamos el Gradient Boosting Regresion

In [13]:
# Modelo de regresión
gbm_reg = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=4,
    random_state=42
)

print("\n🔄 Entrenando modelo de regresión...")
gbm_reg.fit(X_train_reg, y_train_reg)

# Predicciones
y_pred_reg = gbm_reg.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)

print(f"✅ Error cuadrático medio: {mse:.3f}")



🔄 Entrenando modelo de regresión...
✅ Error cuadrático medio: 134.586


## 9) Importancia de características

In [14]:
# INTERPRETABILIDAD - ¿QUÉ CARACTERÍSTICAS SON IMPORTANTES?

print("\n ¿QUÉ CARACTERÍSTICAS SON MÁS IMPORTANTES?")
print("-" * 50)

# Para clasificación (spam)
importance_clf = gbm_clf.feature_importances_
print("📧 Para detectar SPAM:")
for i, imp in enumerate(importance_clf):
    print(f"   {feature_names[i]:20s}: {imp:.3f}")

# Para regresión (casas)
importance_reg = gbm_reg.feature_importances_
print("\n🏠 Para predecir PRECIOS:")
for i, imp in enumerate(importance_reg):
    print(f"   {house_features[i]:20s}: {imp:.3f}")



 ¿QUÉ CARACTERÍSTICAS SON MÁS IMPORTANTES?
--------------------------------------------------
📧 Para detectar SPAM:
   palabras_sospechosas: 0.035
   signos_exclamacion  : 0.136
   enlaces_externos    : 0.000
   mayusculas_porcentaje: 0.780
   longitud_email      : 0.048

🏠 Para predecir PRECIOS:
   metros_cuadrados    : 0.412
   num_habitaciones    : 0.007
   antiguedad          : 0.525
   distancia_centro    : 0.057


## 10) Comparativa

In [15]:
#COMPARACIÓN CON RANDOM FOREST

print("\n GRADIENT BOOSTING vs RANDOM FOREST")
print("-" * 50)

from sklearn.ensemble import RandomForestClassifier

# Entrenar Random Forest para comparar
rf = RandomForestClassifier(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)

print(f"🎯 Gradient Boosting: {accuracy:.3f}")
print(f"🌳 Random Forest:     {rf_accuracy:.3f}")


 GRADIENT BOOSTING vs RANDOM FOREST
--------------------------------------------------
🎯 Gradient Boosting: 0.940
🌳 Random Forest:     0.943


## 11) Conclusion y Consejos Prácticos

📝 PARÁMETROS CLAVE:
   * n_estimators: Más árboles = mejor, pero cuidado con sobreajuste
   * learning_rate: Entre 0.01-0.3. Más bajo = más lento pero más estable
   * max_depth: 3-8 típicamente. Árboles simples funcionan mejor

⚠️  CUIDADOS:
   * Puede sobreajustarse fácilmente
   * Sensible a outliers
   * Más lento que Random Forest

✅ CUÁNDO USARLO:
   * Cuando necesitas máxima precisión
   * Tienes tiempo para tunear parámetros
   * Los datos están limpios

❌ CUÁNDO NO USARLO:
   * Datos muy ruidosos
   * Necesitas resultados rápidos
   * Dataset muy pequeño